<a href="https://colab.research.google.com/github/vindruid/image_quality/blob/master/devina_cloths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Change here, fill with your google drive folder structure
import os

folder_ori    = os.path.join("devina", "original")
folder_rm     = os.path.join("devina", "removed")
folder_result = os.path.join("devina", "result_transparant")

In [0]:
import numpy as np

import cv2

import matplotlib.pyplot as plt
from tqdm import tqdm

from google.colab import drive

def resize_transparant(path_image_ori, path_image_rm):

  ori = cv2.imread(path_image_ori)
  ori = cv2.cvtColor(ori, cv2.COLOR_BGR2BGRA)
  H_ori, W_ori = ori.shape[:2]
  potrait_ori = H_ori > W_ori # H > W

  removed = cv2.imread(path_image_rm, cv2.IMREAD_UNCHANGED)
  H_rm, W_rm = removed.shape[:2]
  potrait = H_rm > W_rm # H > W
  
  if not potrait == potrait_ori: # one potrait, one landscape
    ori = cv2.rotate(ori, cv2.ROTATE_90_CLOCKWISE)
    H_ori, W_ori = ori.shape[:2]

  resized = cv2.resize(removed, (W_ori, H_ori), interpolation = cv2.INTER_AREA) #w,h

  ori_removed = ori.copy()
  ori_removed[:,:,3] = resized[:,:,3]

  return ori_removed

path_drive = os.path.join("drive", "My Drive")

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Define paths
path_ori = os.path.join(path_drive, folder_ori)
path_rm = os.path.join(path_drive, folder_rm)
path_result = os.path.join(path_drive, folder_result)
if not os.path.exists(path_result):
  os.mkdir(path_result)
  
# PROCESS the photos 
list_filename_ori = sorted(os.listdir(path_ori))
for filename_ori in tqdm(list_filename_ori):
  file_id = filename_ori.split('.')[0]
  filename_rm = f'{file_id}-removebg-preview.png'
  filename_res = f'{file_id}_result.png'

  path_image_ori = os.path.join(path_ori, filename_ori)
  path_image_rm  = os.path.join(path_rm, filename_rm)
  path_image_res = os.path.join(path_result, filename_res)

  if not os.path.exists(path_image_rm): 
    print(f"{filename_rm} IS NOT FOUND")
    continue

  ori_removed = resize_transparant(path_image_ori, path_image_rm)

  cv2.imwrite(path_image_res, ori_removed)